In [1]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy import stats
import statsmodels.formula.api as smf

In [2]:
data = pd.read_excel('chemitech.xlsx')
data
# in this data, do anova->check the resultwith same problem and run with the
# help of regression analysis

# dummy_variable concept is to be used for regression analysis

,A,B,C
0,58,58,48
1,64,69,57
2,55,71,59
3,66,64,47
4,67,68,49


In [3]:
# conversion of data sets with new indices
data_r = pd.melt(data.reset_index(), id_vars=['index'], value_vars=['A','B','C'])
data_r.columns = ['index','treatments','value']
data_r

,index,treatments,value
0,0,A,58
1,1,A,64
2,2,A,55
3,3,A,66
4,4,A,67
5,0,B,58
6,1,B,69
7,2,B,71
8,3,B,64
9,4,B,68


In [4]:
# here we get anova table from regression analysis
# all means are not equal so reject null hypothesis
model = ols('value ~ C(treatments)',data=data_r).fit()
anova_table = sm.stats.anova_lm(model,typ=1)
anova_table

,df,sum_sq,mean_sq,F,PR(>F)
C(treatments),2.0,520.0,260.000000,9.176471,0.003818
Residual,12.0,340.0,28.333333,NaN,NaN


In [5]:
# regression analysis using dummy_variable
just_dummies = pd.get_dummies(data_r['treatments'])
just_dummies

,A,B,C
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0
5,0,1,0
6,0,1,0
7,0,1,0
8,0,1,0
9,0,1,0


In [6]:
# now we will drop column C and add dummy-variables into the filename step_1
step_1 = pd.concat([data_r, just_dummies], axis=1)
step_1.drop(['treatments','C'], inplace=True, axis=1)
step_1

,index,value,A,B
0,0,58,1,0
1,1,64,1,0
2,2,55,1,0
3,3,66,1,0
4,4,67,1,0
5,0,58,0,1
6,1,69,0,1
7,2,71,0,1
8,3,64,0,1
9,4,68,0,1


In [7]:
# gives error

# A,B are our independent variable
# p-value 0.00382<0.05  so reject null-hypothesis
# 
result = sm.OLS(step_1['value'], sm.add_constant(step_1[['A','B']])).fit()
result.summary()

D:\Anaconda3\lib\site-packages\scipy\stats\_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=15
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  value   R-squared:                       0.605
Model:                            OLS   Adj. R-squared:                  0.539
Method:                 Least Squares   F-statistic:                     9.176
Date:                Sun, 27 Aug 2023   Prob (F-statistic):            0.00382
Time:                        14:31:17   Log-Likelihood:                -44.691
No. Observations:                  15   AIC:                             95.38
Df Residuals:                      12   BIC:                             97.51
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         52.0000      2.380     21.844      0.000      46.813      57.187
A             10.0000      3.367      2.970      0.012       2.665      17.335
B             14.0000      3.367      4.159      0.001       6.665      21.335
==============================================================================
Omnibus:                        3.780   Durbin-Watson:                   2.550
Prob(Omnibus):                  0.151   Jarque-Bera (JB):                1.311
Skew:                          -0.196   Prob(JB):                        0.519
Kurtosis:                       1.606   Cond. No.                         3.73
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [8]:
model = ols('value ~ A+B', data=step_1).fit()
anova_table = sm.stats.anova_lm(model, typ=1)
anova_table

,df,sum_sq,mean_sq,F,PR(>F)
A,1.0,30.0,30.000000,1.058824,0.323779
B,1.0,490.0,490.000000,17.294118,0.001326
Residual,12.0,340.0,28.333333,NaN,NaN


F-Test is used to test the overall significance of the regression model, t test is used to check the individual significance of each independent variable. 